In [38]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from math import ceil
from tqdm import trange
from subprocess import call
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix

In [2]:
df=pd.read_csv('hackathon_data.csv')
df.dtypes

customer_id                   int64
manufacturer_id               int64
society_id                    int64
city_id                       int64
route_id                      int64
store_id                      int64
order_id                      int64
order_date                   object
category_id                   int64
subcategory_id                int64
product_id                    int64
product_quantity              int64
selling_price_per_unit      float64
total_cost                  float64
subscription                  int64
product_addedtobasket_on     object
dtype: object

In [3]:
len(df)

9678979

In [4]:
df=df.dropna(how='any')

In [5]:
len(df)

9678979

In [6]:
df['order_date']=pd.to_datetime(df['order_date'])
df['product_addedtobasket_on']=pd.to_datetime(df['product_addedtobasket_on'])
df.dtypes

customer_id                          int64
manufacturer_id                      int64
society_id                           int64
city_id                              int64
route_id                             int64
store_id                             int64
order_id                             int64
order_date                  datetime64[ns]
category_id                          int64
subcategory_id                       int64
product_id                           int64
product_quantity                     int64
selling_price_per_unit             float64
total_cost                         float64
subscription                         int64
product_addedtobasket_on    datetime64[ns]
dtype: object

In [7]:
df1=df.drop(['order_date','product_addedtobasket_on','selling_price_per_unit','total_cost'],axis=1)
df1

customer_id  manufacturer_id  society_id  city_id  route_id  \
0            2698080          1122016     1127168  1120112   1121456   
1            1134224          1134336     1120224  1120112   1120336   
2            3686704          1150128     1126160  1120112   1123472   
3            4045328          1122016     1137360  1120112   1121456   
4            3568656          1122016     1120784  1120112   1122576   
5            3641008          1122016     1127168  1120112   1121456   
6            4921952          1122016     1136016  1120112   1123024   
7            2461312          1122016     1121232  1120112   1122688   
8            1588384          1134896     1121568  1120112   1122576   
9            3058944          1126048     1122128  1120112   1121120   
10           3892112          1148896     1120896  1120112   1120448   
11           3181696          1134336     1127840  1120112   1121680   
12           4082176          1150128     1141056  1120112   1121904   
13           2706928          1134336     1127504  1120112   1121792   
14           4615296          1150128     1156624  1120112   1123920   
15           1577968          1150128     1123360  1120112   1121456   
16           1504720          1150128     1121680  1120112   1121120   
17           4034688          1122016     1127168  1120112   1121456   
18           3580416          1122016     1129856  1120112   1122352   
19           2371264          1122016     1127504  1120112   1121792   
20           1540448          1124592     1123248  1120112   1120672   
21           4469696          1150128     1126048  1120112   1123472   
22           4643744          1122016     1120112  1120112   1120224   
23           4275600          1128288     1123360  1120112   1121456   
24           4469696          1150128     1126048  1120112   1123472   
25           1712816          1150128     1123136  1120112   1124144   
26           1777328          1122016     1122464  1120112   1120896   
27           3843840          1136464     1120112  1120112   1120224   
28           2548000          1186528     1130864  1120112   1122128   
29           3749648          1149120     1160992  1120112   1123472   
...              ...              ...         ...      ...       ...   
9678949     37245040          1188768     1226288  1120560   1132656   
9678950     20567568          1122016     1184960  1120224   1130080   
9678951     14717808          1122016     1201760  1120560   1132656   
9678952     23484384          1124368     1201536  1120560   1151024   
9678953     27410880          1122016     1201424  1120560   1151024   
9678954     20213872          1150128     1188544  1120224   1129520   
9678955     33721520          1150128     1185408  1120224   1129968   
9678956      4539024          1123472     1330560  1120224   1143072   
9678957     31154144          1150128     1200192  1120560   1130528   
9678958     23462880          1136352     1222144  1120560   1142624   
9678959     12552064          1150128     1184848  1120224   1129744   
9678960     24032736          1150128     1222928  1120560   1130976   
9678961     18247600          1150128     1202432  1120560   1139936   
9678962     30832704          1150128     1189888  1120224   1129184   
9678963     14162288          1130528     1189664  1120224   1131536   
9678964     23741872          1122016     1191568  1120224   1156960   
9678965      3498320          1122016     1187200  1120224   1158864   
9678966     16035824          1134336     1194480  1120224   1156176   
9678967     11456816          1129072     1194592  1120224   1156064   
9678968     13232464          1129072     1192464  1120224   1157184   
9678969     11602640          1150128     1187088  1120224   1158976   
9678970     13354656          1150128     1191008  1120224   1156848   
9678971     11029312          1134336     1187088  1120224   1158976   
9678972     29033536          1134336     1192016

In [8]:
df2=df1[['customer_id','product_id','product_quantity']]
df2.head()

customer_id  product_id  product_quantity
0      2698080     1120336                 4
1      1134224     1939280                 1
2      3686704     1681456                 1
3      4045328     1120224                 3
4      3568656     1120112                 3

In [9]:
df2=df2.groupby(['customer_id','product_id']).sum()
df2=pd.DataFrame(df2)
df2.reset_index()

customer_id  product_id  product_quantity
0            1121904     1120224                17
1            1121904     1120336                52
2            1121904     1120672                 1
3            1121904     1120896                11
4            1121904     1124256                 7
5            1121904     1133216                 4
6            1121904     1360800                 1
7            1121904     1367072                 1
8            1121904     1379840                 1
9            1121904     1414672                 4
10           1121904     1422960                 4
11           1121904     1518832                 1
12           1121904     1569792                 1
13           1121904     1579648                 1
14           1121904     1645840                 1
15           1121904     1677536                 2
16           1121904     1681904                 2
17           1121904     1682800                 1
18           1121904     1683696                 6
19           1121904     1688064                 4
20           1121904     1688960                 7
21           1121904     1694112                 2
22           1121904     1709008                 4
23           1121904     1728608                 1
24           1121904     1745856                 1
25           1121904     1853488                 3
26           1121904     1870400                 1
27           1121904     1902656                 2
28           1121904     1903888                 2
29           1121904     1908928                 2
...              ...         ...               ...
2802167     48171648     2432080                 2
2802168     48171648     2452912                 1
2802169     48171648     2507120                 3
2802170     48171648     2547104                 1
2802171     48171648     2547328                 1
2802172     48171648     2705584                 1
2802173     48171648     2716896                 1
2802174     48171648     2718016                 1
2802175     48171648     2722720                 1
2802176     48171648     2842448                 2
2802177     48171648     2901584                 1
2802178     48171648     2929696                 1
2802179     48171648     3192448                 1
2802180     48171648     3352832                 1
2802181     48171648     3352944                 1
2802182     48171648     3353056                 2
2802183     48171648     3410960                 2
2802184     48171648     3411072                 7
2802185     48171648     3470432                 1
2802186     48171648     3621632                 1
2802187     48171648     3741136                 2
2802188     48171648     3784928                 1
2802189     48171648     3816960                 4
2802190     48171648     3939264                 1
2802191     48171648     3948896                 1
2802192     48171648     4083632                 1
2802193     48171648     4125520                 2
2802194     48171648     4125632                 1
2802195     48171648     4135824                 1
2802196     48171648     4164608                 1

[2802197 rows x 3 columns]

In [10]:
df_matrix = pd.pivot_table(df2, values='product_quantity', index='customer_id', columns='product_id')
df_matrix

product_id   1120112  1120224  1120336  1120448  1120560  1120672  1120784  \
customer_id                                                                  
1121904          NaN     17.0     52.0      NaN      NaN      1.0      NaN   
1122016        108.0    262.0      NaN      NaN      NaN      NaN      NaN   
1122128          NaN      NaN    251.0      NaN      NaN      NaN      NaN   
1122352          NaN     98.0      NaN      NaN      NaN      NaN      NaN   
1123248          NaN    519.0      NaN      NaN      NaN      NaN      NaN   
1123696          8.0    178.0      NaN      NaN      NaN      NaN      NaN   
1124144          4.0    917.0      NaN      NaN      NaN      NaN      NaN   
1124256          NaN     55.0      NaN      NaN     74.0    102.0      1.0   
1124592          NaN   1324.0      NaN      NaN      NaN      NaN      NaN   
1126608         28.0    969.0      NaN      NaN      NaN      NaN      8.0   
1128400        451.0      1.0     95.0      NaN      NaN      NaN      NaN   
1128512          NaN   1707.0      6.0      NaN      NaN    110.0      1.0   
1129072          NaN      NaN      5.0      NaN      2.0      NaN      NaN   
1129296          NaN      NaN    910.0     35.0      NaN      NaN      2.0   
1132320          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1133888        363.0     58.0      NaN      NaN      NaN      NaN      NaN   
1134000       2238.0     47.0      NaN      NaN      NaN      NaN      NaN   
1134224          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1136576          2.0     35.0      NaN      NaN    246.0      NaN      NaN   
1136688          NaN    476.0      NaN      NaN      NaN      NaN      NaN   
1139376          NaN      2.0      4.0      2.0      6.0      NaN      NaN   
1139824         77.0    598.0      NaN      NaN      NaN      NaN      NaN   
1139936          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1140160          NaN    257.0      NaN      NaN      NaN      NaN     32.0   
1140384          NaN    887.0      NaN      NaN      NaN      NaN      1.0   
1140608          NaN      NaN      2.0      8.0      3.0    323.0      NaN   
1141168         75.0   1669.0      NaN      NaN      NaN      NaN      NaN   
1142176          NaN   1566.0      NaN      NaN      NaN      NaN      NaN   
1142848          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1143520         40.0    799.0      NaN      NaN      NaN      NaN      NaN   
...              ...      ...      ...      ...      ...      ...      ...   
45300080         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
45323040         2.0      8.0      NaN      3.0     38.0      NaN      NaN   
45354848         7.0      NaN      NaN    110.0    101.0      NaN      NaN   
45427760        60.0     83.0      NaN      2.0      NaN      NaN      NaN   
45442880         NaN     56.0      NaN      NaN      3.0      NaN      NaN   
45532928         8.0      4.0      NaN     33.0     63.0      9.0      NaN   
45553088         NaN      2.0      NaN      NaN      NaN      NaN      NaN   
45576496         NaN      NaN      NaN      NaN      NaN      NaN      1.0   
45644704         1.0      8.0      NaN      1.0      2.0      NaN      NaN   
45725008        11.0      NaN      NaN      NaN      NaN      NaN      1.0   
45843840         NaN      1.0     24.0      NaN      NaN      NaN      NaN   
45856608         NaN      NaN      NaN      NaN      NaN    111.0      1.0   
45891664        77.0     16.0      NaN      NaN      NaN      NaN      NaN   
45894016        39.0      1.0      NaN     13.0      NaN      NaN      1.0   
45906896         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
45966592         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
45987200         2.0      NaN      NaN      NaN      NaN      2.0      NaN   
46036032         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
46215232         NaN  

In [12]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [75]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head(20)

(2757824, 3)


customer_id product_id  scaled_purchase_freq
1       1122016    1120112              0.047661
5       1123696    1120112              0.003118
6       1124144    1120112              0.001336
9       1126608    1120112              0.012027
10      1128400    1120112              0.200445
15      1133888    1120112              0.161247
16      1134000    1120112              0.996437
18      1136576    1120112              0.000445
21      1139824    1120112              0.033853
26      1141168    1120112              0.032962
29      1143520    1120112              0.017372
30      1144080    1120112              0.438307
34      1147664    1120112              0.003563
35      1150464    1120112              0.001782
36      1150912    1120112              0.076169
39      1153824    1120112              0.000000
42      1155504    1120112              0.231180
43      1155840    1120112              0.000000
46      1156624    1120112              0.255234
47      1158640    1120112              0.014254

In [111]:
len(data_norm['customer_id'].unique())

30000

In [76]:
def create_matrix(data, users_col, items_col, ratings_col, threshold = None):
    if threshold is not None:
        data = data[data[ratings_col] >= threshold]
        data[ratings_col] = 1
    else:
        data[ratings_col]=-1
    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].astype('category')

    ratings = csr_matrix((data[ratings_col],
                          (data[users_col].cat.codes, data[items_col].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data

In [83]:
items_col = 'product_id'
users_col = 'customer_id'
ratings_col = 'scaled_purchase_freq'
threshold = 0.3
X, df = create_matrix(data_norm, users_col, items_col, ratings_col, threshold)
X

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<22146x7911 sparse matrix of type '<class 'numpy.int64'>'
	with 75823 stored elements in Compressed Sparse Row format>

In [51]:
class BPR:
    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 batch_size = 1000, reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
    def fit(self, ratings):
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape
        
        # ensure batch size makes sense, since the algorithm involves
        # for each step randomly sample a user, thus the batch size
        # should be smaller than the total number of users or else
        # we would be sampling the user with replacement
        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write('WARNING: Batch size is greater than number of users,'
                             'switching to a batch size of {}\n'.format(n_users))

        batch_iters = n_users // batch_size
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size = (n_users, self.n_factors))
        self.item_factors = rstate.normal(size = (n_items, self.n_factors))
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_users, n_items, indices, indptr)
                sampled_users, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)

        return self
        
    def _sample(self, n_users, n_items, indices, indptr):
        """sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_neg_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_users = np.random.choice(
            n_users, size = self.batch_size, replace = False)

        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user]:indptr[user + 1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)

            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items
    def _update(self, u, i, j):
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]
        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid n_factors times so
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T

        # update using gradient descent
        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg * user_u
        grad_i = sigmoid_tiled * -user_u + self.reg * item_i
        grad_j = sigmoid_tiled * user_u + self.reg * item_j
        self.user_factors[u] -= self.learning_rate * grad_u
        self.item_factors[i] -= self.learning_rate * grad_i
        self.item_factors[j] -= self.learning_rate * grad_j
        return self
    
    def predict(self):
        if self._prediction is None:
            self._prediction = self.user_factors.dot(self.item_factors.T)

        return self._prediction
    
    def _predict_user(self, user):
        user_pred = self.user_factors[user].dot(self.item_factors.T)
        return user_pred
   
    def recommend(self, ratings, N = 5):
        n_users = ratings.shape[0]
        recommendation = np.zeros((n_users, N), dtype = np.uint32)
        for user in range(n_users):
            top_n = self._recommend_user(ratings, user, N)
            recommendation[user] = top_n

        return recommendation

    def _recommend_user(self, ratings, user, N):
        """the top-N ranked items for a given user"""
        scores = self._predict_user(user)

        
        liked = set(ratings[user].indices)
        count = N + len(liked)
        if count < scores.shape[0]:

            
            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
        else:
            best = np.argsort(scores)[::-1]

        top_n = list(islice((rec for rec in best if rec not in liked), N))
        return top_n
    
    def get_similar_items(self, N = 5, item_ids = None):
        
        normed_factors = normalize(self.item_factors)
        knn = NearestNeighbors(n_neighbors = N + 1, metric = 'euclidean')
        knn.fit(normed_factors)

       
        if item_ids is not None:
            normed_factors = normed_factors[item_ids]

        _, items = knn.kneighbors(normed_factors)
        similar_items = items[:, 1:].astype(np.uint32)
        return similar_items


                    
    

In [52]:
def create_train_test(ratings, test_size = 0.4, seed = 1234):
    
    assert test_size < 1.0 and test_size > 0.0

    
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)
    
   
    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size = n_splits, replace = False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0
    
    train, test = train.tocsr(), test.tocsr()
    return train, test

In [87]:

X_train, X_test = create_train_test(X, test_size = 0.4, seed = 1234)

In [89]:
bpr_params = {'reg': 0.01,
              'learning_rate': 0.1,
              'n_iters': 160,
              'n_factors': 15,
              'batch_size': 10000}

bpr = BPR(**bpr_params)
bpr.fit(X_test)




BPR:   0%|          | 0/160 [00:00<?, ?it/s]


BPR:   1%|          | 1/160 [00:00<00:46,  3.44it/s]


BPR:   1%|▏         | 2/160 [00:00<00:46,  3.43it/s]


BPR:   2%|▏         | 3/160 [00:00<00:45,  3.42it/s]


BPR:   2%|▎         | 4/160 [00:01<00:45,  3.42it/s]


BPR:   3%|▎         | 5/160 [00:01<00:45,  3.42it/s]


BPR:   4%|▍         | 6/160 [00:01<00:45,  3.42it/s]


BPR:   4%|▍         | 7/160 [00:02<00:44,  3.42it/s]


BPR:   5%|▌         | 8/160 [00:02<00:44,  3.42it/s]


BPR:   6%|▌         | 9/160 [00:02<00:49,  3.07it/s]


BPR:   6%|▋         | 10/160 [00:03<00:55,  2.69it/s]


BPR:   7%|▋         | 11/160 [00:03<00:53,  2.77it/s]


BPR:   8%|▊         | 12/160 [00:03<00:56,  2.61it/s]


BPR:   8%|▊         | 13/160 [00:04<00:57,  2.57it/s]


BPR:   9%|▉         | 14/160 [00:04<00:54,  2.69it/s]


BPR:   9%|▉         | 15/160 [00:05<00:50,  2.89it/s]


BPR:  10%|█         | 16/160 [00:05<00:48,  2.97it/s]


BPR:  11%|█         | 17/160 [00:05<00:46,  3.06it/s]


BPR:  1

In [91]:
bpr.get_similar_items(N = 5)

array([[5043, 3541, 2732, 4380, 6150],
       [2094, 5876, 5416,  522, 1475],
       [3111, 4631, 3260, 3599, 6209],
       ...,
       [4647, 4904, 6285, 2928, 5071],
       [ 957, 4548, 4451, 3572, 7742],
       [6334,   84, 4911, 4179, 3025]], dtype=uint32)

In [235]:
z=bpr.recommend(X_test, N = 10)
z=pd.DataFrame(z)



In [252]:
r=pd.DataFrame(data_norm['product_id'].unique())
q=data_norm['customer_id'].unique()
z.index=q[0:22146]
z
r['index']=r.index
r['product_id']=r[0]
r.drop(['index',0],axis=1,inplace=True)

In [261]:
z=z.replace({0:r.product_id,1:r.product_id,2:r.product_id,3:r.product_id,4:r.product_id,5:r.product_id,6:r.product_id,7:r.product_id,8:r.product_id,9:r.product_id})


0    1694000
1    1548512
2    3407376
3    1681680
4    2158464
5    2159360
6    1785168
7    2039408
8    2839648
9    3411184
Name: 4921952, dtype: int64

In [114]:
def auc_score(model, ratings):
    auc = 0.0
    n_users, n_items = ratings.shape
    for user, row in enumerate(ratings):
        y_pred = model._predict_user(user)
        y_true = np.zeros(n_items)
        y_true[row.indices] = 1
        auc += roc_auc_score(y_true, y_pred)

    auc /= n_users
    return auc
print(auc_score(bpr, X_test))

0.991848118760626


In [120]:
df7=pd.read_csv('hackathon_data.csv',)

In [121]:
len(df7['category_id'].unique())

22

In [124]:
df7=df7[['category_id','product_addedtobasket_on']]
df7['product_addedtobasket_on']=pd.to_datetime(df7['product_addedtobasket_on'])


category_id                          int64
product_addedtobasket_on    datetime64[ns]
dtype: object

In [131]:
df7['category_id']=df7['category_id'].astype('category')

In [162]:
df8=pd.get_dummies(df7)
columns=[df7['category_id'].unique()]
df8.set_index('product_addedtobasket_on')


category_id_1120000  category_id_1122352  \
product_addedtobasket_on                                             
2018-04-30 16:23:46                         0                    0   
2018-04-19 01:30:02                         0                    0   
2018-04-30 22:26:07                         0                    0   
2018-04-23 01:30:03                         0                    0   
2018-04-30 22:22:57                         0                    0   
2018-04-25 01:30:03                         0                    0   
2018-04-30 10:00:47                         0                    0   
2018-04-30 22:33:59                         0                    0   
2018-04-20 01:30:03                         0                    0   
2018-04-30 22:20:26                         0                    0   
2018-04-30 19:57:02                         0                    0   
2018-04-30 08:59:05                         0                    0   
2018-04-30 20:14:56                         0                    0   
2018-04-30 19:31:10                         0                    0   
2018-04-30 13:27:38                         0                    0   
2018-04-30 22:31:12                         0                    0   
2018-04-30 17:25:58                         0                    0   
2018-04-22 01:30:02                         0                    0   
2018-04-19 01:30:02                         0                    0   
2018-04-24 01:30:02                         0                    0   
2018-04-30 19:04:52                         0                    0   
2018-04-30 16:54:38                         0                    0   
2018-04-24 19:12:49                         0                    0   
2018-04-30 21:39:21                         0                    0   
2018-04-30 16:54:07                         0                    0   
2018-04-30 20:47:08                         0                    0   
2018-04-23 01:30:03                         0                    0   
2018-04-30 19:34:04                         0                    0   
2018-04-30 15:23:24                         0                    0   
2018-04-30 22:55:48                         0                    0   
...                                       ...                  ...   
2019-07-30 19:49:46                         0                    0   
2019-07-21 01:30:02                         0                    0   
2019-07-23 01:30:02                         0                    0   
2019-07-30 21:40:53                         0                    0   
2019-07-30 21:12:44                         0                    0   
2019-07-30 10:13:34                         0                    0   
2019-07-30 22:55:00                         0                    0   
2019-07-30 18:39:41                         0                    0   
2019-07-30 22:52:35                         0                    0   
2019-07-30 22:58:45                         0                    0   
2019-07-30 23:41:39                         0                    0   
2019-07-30 22:02:58                         0                    0   
2019-07-30 22:00:55                         0                    0   
2019-07-30 19:50:44                         0                    0   
2019-07-30 22:14:58                         0                    0   
2019-07-21 01:30:02                         0                    0   
2019-07-19 01:30:02                         0                    0   
2019-07-22 01:30:01                         0                    0   
2019-07-30 10:41:36                         0                    0   
2019-07-30 23:27:24                         0                    0   
2019-07-30 23:02:56                         0                    0   
2019-07-30 17:55:32                         0                    0   
2019-07-21 01:30:02                         0                    0   
2019-07-30 22:50:49                         0                    0   
2019-07-30 23:15:53                 

In [186]:
df9=df8.drop(['product_addedtobasket_on'],axis=1)

In [187]:
df9.index=df8.product_addedtobasket_on

In [188]:
train = df9[:int(0.8*(len(df9)))]
valid = df9[int(0.8*(len(df9))):]

#Fit the model
from statsmodels.tsa.vector_ar.var_model import VAR
model = VAR(endog=train)
model_fit = model.fit()

#make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))


/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [ ]:
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,22):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(pred[i], valid[i])))

<module 'pandas' from '/anaconda3/lib/python3.7/site-packages/pandas/__init__.py'>